In [1]:
import pandas as pd
import numpy as np
from langdetect import detect
import re

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
comments = pd.read_csv('../data/comments_lang.csv')
comments_en = comments[comments.lang == 'en']
vect = CountVectorizer(ngram_range = (1,1), analyzer = 'word',
                       stop_words = 'english',
                       max_features = 10000,
                       min_df = 2, max_df = 0.95).fit(comments_en.Review)

In [3]:
pw = list(vect.vocabulary_.keys())

In [4]:
import enchant
c = enchant.Dict("en_UK")
def check_spelling(text):
    if not c.check(text):
        suggestions = list(set(c.suggest(text)).intersection(set(pw)))
        if len(suggestions)>0:
            res = suggestions[0]
        else:
            res = text
    else:
        res = text
    return res

In [5]:
check_spelling('stu pid')

'stupid'

In [6]:
labeled1 = pd.read_excel('manual_labels.xlsx')
labeled2 = pd.read_excel('manual_labels2.xlsx')
labeled3 = pd.read_excel('manual_labels3.xlsx')
labeled4 = pd.read_excel('manual_labels4.xlsx')
labeled5 = pd.read_excel('manual_labels5.xlsx')

In [7]:
labeled5.label.value_counts()

4    621
2    172
5     89
0     82
3     25
1     11
Name: label, dtype: int64

In [8]:
labeled2.shape

(1000, 5)

In [9]:
labeled1.label.value_counts()

4    624
2    203
5     74
0     60
3     31
1      8
Name: label, dtype: int64

In [10]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
def clean_comment(text):
    wnl = WordNetLemmatizer()
    deacc = re.sub(r'\W',' ', text)
    tokens = word_tokenize(deacc)
    res = ''
    for t in tokens:
        res += wnl.lemmatize(t)+' '
    return res
def get_tokens(text):
    wnl = WordNetLemmatizer()
    deacc = re.sub(r'\W',' ', text)
    tokens = word_tokenize(deacc)
    return tokens

In [11]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
import nltk
from nltk.corpus import wordnet
from emoji.unicode_codes import UNICODE_EMOJI
import emoji
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

def clean_comment(text):
    wnl = WordNetLemmatizer()
    deacc = re.sub(r'\!',' exclamation_point ', text)
    tokens = word_tokenize(deacc)
    tags = nltk.pos_tag(tokens)
    processed = []
    for (word, tag) in tags:
        wn_tag = get_wordnet_pos(tag)
        if wn_tag!='':
            processed.append(wnl.lemmatize(word,wn_tag))
        else:
            processed.append(wnl.lemmatize(check_spelling(word)))
    res = ' '.join(processed)
    return res

In [12]:
labeled = pd.concat([labeled1, labeled2, labeled3, labeled4, labeled5],axis = 0)

In [13]:
labeled.loc[:,'cleaned'] = labeled.Review.apply(clean_comment)

In [14]:
labeled['tokens'] = labeled.Review.apply(get_tokens)

In [15]:
labeled_long = labeled[labeled.tokens.apply(len)>6]
labeled_neg = labeled[labeled.label!=4]

In [16]:
#labeled[labeled.label==1].to_excel('labeled_crash0.xlsx')

In [17]:
#comments_en['cleaned'] = comments_en.Review.apply(clean_comment)

In [18]:
#comments_en.to_csv('comments_en_cleaned.csv')

In [19]:
comments_en = pd.read_csv('comments_en_cleaned.csv')

In [20]:
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

In [21]:
def eval_model(y_train,y_test,y_train_pred,y_test_pred):
    
    class_names = ['unknown',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']
    
    class_names_b = ['neg', 'pos']
    print('train scores\n')
    print(classification_report(y_train, y_train_pred, target_names = class_names))
    print('test scores\n')
    print(classification_report(y_test, y_test_pred, target_names = class_names))

In [22]:
vect = CountVectorizer(ngram_range = (1,3), analyzer = 'word',
                       stop_words = 'english',
                       max_features = 10000,
                       min_df = 2, max_df = 0.95).fit(comments_en.cleaned)
vocab = vect.vocabulary_

In [24]:
from sklearn.pipeline import Pipeline
vect = CountVectorizer()
model = RidgeClassifier()
lin_model = Pipeline([('vectorizer', vect), ('classifier', model)])
lin_model.set_params(vectorizer__ngram_range = (1,3),vectorizer__analyzer = 'word', vectorizer__stop_words = 'english',
                     vectorizer__max_features = 800,
                     vectorizer__min_df = 2, vectorizer__max_df = 0.95,
                     vectorizer__vocabulary = vocab,
                     
                     classifier__class_weight = 'balanced', classifier__alpha = 0.1)
feats = labeled.cleaned
labels = labeled.label
X_train, X_test, y_train, y_test = train_test_split(feats, labels, test_size=0.2)
lin_model = lin_model.fit(X_train, y_train)
y_train_pred = lin_model.predict(X_train)
y_test_pred = lin_model.predict(X_test)
eval_model(y_train,y_test,y_train_pred,y_test_pred)
print('cross_val_score: %1.4f'% (np.mean(cross_val_score(lin_model, labeled.cleaned,labeled.label))))

train scores

                  precision    recall  f1-score   support

         unknown       0.59      0.72      0.65       299
           Crash       0.14      1.00      0.24        36
Balance problems       0.67      0.68      0.68       769
 Synchronization       0.53      0.94      0.68       116
        Positive       0.93      0.74      0.82      2453
             Bug       0.56      0.76      0.65       327

     avg / total       0.81      0.74      0.76      4000

test scores

                  precision    recall  f1-score   support

         unknown       0.49      0.68      0.57        63
           Crash       0.06      0.50      0.11        10
Balance problems       0.71      0.68      0.70       203
 Synchronization       0.47      0.71      0.56        38
        Positive       0.92      0.71      0.80       603
             Bug       0.40      0.53      0.46        83

     avg / total       0.78      0.69      0.72      1000



/Users/dns/anaconda/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


cross_val_score: 0.4676


In [25]:
from sklearn.metrics import f1_score,roc_auc_score,accuracy_score
def eval_classifier(input_text,model):
    cleaned_text = clean_comment(input_text)
    feats = vect.transform([cleaned_text])
    class_names = ['Other',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']
    prediction = model.predict(feats.toarray())
    #print(class_names[prediction[0]])
    return prediction[0]
def eval_pipeline(input_text, model = lin_model):
    cleaned_text = clean_comment(input_text)
    class_names = ['Other',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']
    prediction = model.predict([cleaned_text])
    #print(class_names[prediction[0]])
    return prediction[0]
def val_score(model):
    class_names = ['Other',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']    
    val_en = pd.read_excel('validation_en.xlsx')
    y_true = []
    y_pred = []
    for i in range(0,6):
        y_true.append([i]*10)
        y_pred.append(val_en.iloc[:,i].apply(eval_pipeline))
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()
    print(classification_report(y_true, y_pred, target_names = class_names))
    print('model accuracy %1.4f'%(accuracy_score(y_true, y_pred)))
    return y_true,y_pred
y_true,y_pred = val_score(lin_model)

                  precision    recall  f1-score   support

           Other       1.00      0.90      0.95        10
           Crash       0.62      0.50      0.56        10
Balance problems       0.56      0.90      0.69        10
 Synchronization       0.73      0.80      0.76        10
        Positive       0.86      0.60      0.71        10
             Bug       0.78      0.70      0.74        10

     avg / total       0.76      0.73      0.73        60

model accuracy 0.7333


In [26]:
eval_pipeline('this dum gaem crashes every time i launch it')

1

In [27]:
eval_pipeline('those new weapons are so dam op')

2

In [28]:
eval_pipeline('The game glitched and all of my trophies and guns are now gone')

3

In [29]:
eval_pipeline('Cool!')

4

In [30]:
eval_pipeline('This game is haard to control')

5

In [38]:
labeled_crash = pd.read_excel('labeled_crash0.xlsx')

In [39]:
labeled_crash.subcat.value_counts()

2    25
1    14
0     7
Name: subcat, dtype: int64

In [44]:
from sklearn.pipeline import Pipeline
vect = CountVectorizer()
model = LogisticRegression()
lin_model = Pipeline([('vectorizer', vect), ('classifier', model)])
lin_model.set_params(vectorizer__ngram_range = (1,3),vectorizer__analyzer = 'word', vectorizer__stop_words = 'english',
                     vectorizer__max_features = 800,
                     vectorizer__min_df = 2, vectorizer__max_df = 0.95,
                     vectorizer__vocabulary = vocab,
                     
                     classifier__class_weight = 'balanced', classifier__C = 0.1)
feats = labeled_crash.cleaned
labels = labeled_crash.subcat
X_train, X_test, y_train, y_test = train_test_split(feats, labels, test_size=0.2)
lin_model = lin_model.fit(X_train, y_train)
y_train_pred = lin_model.predict(X_train)
y_test_pred = lin_model.predict(X_test)
eval_model(y_train,y_test,y_train_pred,y_test_pred)
print('cross_val_score: %1.4f'% (np.mean(cross_val_score(lin_model, labeled.cleaned,labeled.label))))

train scores

                  precision    recall  f1-score   support

         unknown       1.00      1.00      1.00         3
           Crash       1.00      0.75      0.86        12
Balance problems       0.88      1.00      0.93        21

     avg / total       0.93      0.92      0.91        36

test scores

                  precision    recall  f1-score   support

         unknown       0.00      0.00      0.00         4
           Crash       1.00      0.50      0.67         2
Balance problems       0.44      1.00      0.62         4

     avg / total       0.38      0.50      0.38        10



/Users/dns/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 6
  .format(len(labels), len(target_names))
/Users/dns/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


cross_val_score: 0.7876


In [46]:
from sklearn.metrics import f1_score,roc_auc_score,accuracy_score
def eval_classifier(input_text,model):
    cleaned_text = clean_comment(input_text)
    feats = vect.transform([cleaned_text])
    class_names = ['Other',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']
    prediction = model.predict(feats.toarray())
    #print(class_names[prediction[0]])
    return prediction[0]
def eval_pipeline(input_text, model = lin_model):
    cleaned_text = clean_comment(input_text)
    class_names = ['Other',
        'Load',
        'In game']
    prediction = model.predict([cleaned_text])
    #print(class_names[prediction[0]])
    return prediction[0]
def val_score(model):
    class_names = ['Other',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']    
    val_en = pd.read_excel('validation_en.xlsx')
    y_true = []
    y_pred = []
    for i in range(0,6):
        y_true.append([i]*10)
        y_pred.append(val_en.iloc[:,i].apply(eval_pipeline))
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()
    print(classification_report(y_true, y_pred, target_names = class_names))
    print('model accuracy %1.4f'%(accuracy_score(y_true, y_pred)))
    return y_true,y_pred
y_true,y_pred = val_score(lin_model)

                  precision    recall  f1-score   support

           Other       0.00      0.00      0.00        10
           Crash       0.67      0.20      0.31        10
Balance problems       0.18      1.00      0.30        10
 Synchronization       0.00      0.00      0.00        10
        Positive       0.00      0.00      0.00        10
             Bug       0.00      0.00      0.00        10

     avg / total       0.14      0.20      0.10        60

model accuracy 0.2000


/Users/dns/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
eval_pipeline()